In [2]:
import os
import re
import csv
import glob
import operator
import collections
import numpy as np
from collections import Counter


In [3]:
# split word to ( , )

train_dir = 'keywordmatrix'

name_list = os.listdir(train_dir)
full_list = [os.path.join(train_dir,i) for i in name_list]
time_sorted_list = sorted(full_list, key=os.path.getctime)
print(time_sorted_list[-1])

all_words = []
for email in time_sorted_list:
    with open(email, encoding = 'ISO-8859-1') as m:
        a = []
        for i,line in enumerate(m):
            if i != 1:
                words_ = line
                pattern3 = re.compile("[^\w\d]+")
                words_ = pattern3.sub(' ',words_)
                words = words_.split()
                for j in range(len(words)):
                    a.append(words[j])
    all_words.append(a)
print ('size of letter :', len(all_words))

keywordmatrix\0001.1999-12-10.farmer.ham.txt
size of letter : 254


In [4]:
import numpy as np

np.save('keywordmatrix.npy', all_words)

In [5]:
cut_programs = np.load('keywordmatrix.npy')

In [6]:
voc_dict = np.load('keyword_voc_dict.npy')

In [7]:
# word filter

example_doc = []

for letter in cut_programs:
    example_line = ''
    for w in letter:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print( example_doc[0] )

com body font 14px 20px Helvetica Arial sans serif margin padding text align center webkit text size adjust none p padding 10px h1 img max 100 height auto vertical align bottom h2 font size 22px line height 28px margin 12px h3 margin 12px background none padding border none wrapper 600px margin auto 10px auto text align left input button border none button display inline block font weight 500 font size 16px line height 42px font family Helvetica Arial sans serif auto white space nowrap height 42px margin 12px 5px 12px padding 22px text decoration none text align center cursor pointer border border radius 3px vertical align top button span display inline font family Helvetica Arial sans serif text decoration none font weight 500 font style normal font size 16px line height 42px cursor pointer border none border radius 6px poweredWrapper padding 20px margin auto display block span display inline block height 32px line height 32px padding 5px margin 5px 5px clear clear both profile list d

### tfidf

In [9]:
import math
import nltk
from textblob import TextBlob as tb

nltk.download('punkt')

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nems\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
bloblist = []
for i in range(len(example_doc)):
    blob = tb(example_doc[i])
    bloblist.append(blob)
print (len(bloblist))

254


In [11]:
kw_n = 10

key_list = []
for i, blob in enumerate(bloblist):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    tmp = ""
    for word, score in sorted_words[ :kw_n]:
        tmp += word + ' '
    key_list.append(tmp)

### word2vec

In [12]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\nems\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
# train word2vec on the two sentences

#model = gensim.models.Word2Vec(example_doc, min_count=1, size=200, hs=1, negative=0)
model = gensim.models.Word2Vec.load("word2vec.sav")

2017-10-29 09:19:17,390 : INFO : loading Word2Vec object from word2vec.sav
2017-10-29 09:19:17,395 : INFO : loading wv recursively from word2vec.sav.wv.* with mmap=None
2017-10-29 09:19:17,396 : INFO : setting ignored attribute syn0norm to None
2017-10-29 09:19:17,399 : INFO : setting ignored attribute cum_table to None
2017-10-29 09:19:17,401 : INFO : loaded word2vec.sav


In [14]:
letter_score = []
for letter in key_list:
    if len(letter) == 0:
        pass
    else:
        score = model.score(letter.split())
        letter_score.append(score.reshape(1, score.shape[0]))

2017-10-29 09:19:17,413 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,425 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,428 : INFO : scoring 10 sentences took 0.0s, 884 sentences/s
2017-10-29 09:19:17,432 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,442 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,444 : INFO : scoring 10 sentences took 0.0s, 903 sentences/s
2017-10-29 09:19:17,446 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,450 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,451 : INFO : scoring 10 sentences took 0.0s, 3153 sentences/s
2017-10-29 09:19:17,453 : INFO : scoring sent

2017-10-29 09:19:17,643 : INFO : scoring 10 sentences took 0.0s, 2645 sentences/s
2017-10-29 09:19:17,646 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,651 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,653 : INFO : scoring 10 sentences took 0.0s, 2505 sentences/s
2017-10-29 09:19:17,655 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,658 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,660 : INFO : scoring 10 sentences took 0.0s, 3047 sentences/s
2017-10-29 09:19:17,662 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,670 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,675 : INFO : scoring 10

2017-10-29 09:19:17,881 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,887 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,889 : INFO : scoring 10 sentences took 0.0s, 1819 sentences/s
2017-10-29 09:19:17,891 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,897 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,902 : INFO : scoring 10 sentences took 0.0s, 1111 sentences/s
2017-10-29 09:19:17,904 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:17,907 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:17,909 : INFO : scoring 10 sentences took 0.0s, 3329 sentences/s
2017-10-29 09:19:17,911 : INFO : scoring se

2017-10-29 09:19:18,098 : INFO : scoring 10 sentences took 0.0s, 2026 sentences/s
2017-10-29 09:19:18,100 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,104 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,106 : INFO : scoring 10 sentences took 0.0s, 2846 sentences/s
2017-10-29 09:19:18,109 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,116 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,118 : INFO : scoring 10 sentences took 0.0s, 1482 sentences/s
2017-10-29 09:19:18,120 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,122 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,124 : INFO : scoring 10

2017-10-29 09:19:18,319 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,322 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,325 : INFO : scoring 10 sentences took 0.0s, 3053 sentences/s
2017-10-29 09:19:18,326 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,331 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,334 : INFO : scoring 10 sentences took 0.0s, 1723 sentences/s
2017-10-29 09:19:18,339 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,341 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,343 : INFO : scoring 10 sentences took 0.0s, 3478 sentences/s
2017-10-29 09:19:18,345 : INFO : scoring se

2017-10-29 09:19:18,543 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,545 : INFO : scoring 10 sentences took 0.0s, 2733 sentences/s
2017-10-29 09:19:18,548 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,555 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,558 : INFO : scoring 10 sentences took 0.0s, 1625 sentences/s
2017-10-29 09:19:18,560 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,562 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,564 : INFO : scoring 10 sentences took 0.0s, 3734 sentences/s
2017-10-29 09:19:18,566 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,569 : INFO : reached en

2017-10-29 09:19:18,770 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,774 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,776 : INFO : scoring 10 sentences took 0.0s, 2965 sentences/s
2017-10-29 09:19:18,778 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,781 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,783 : INFO : scoring 10 sentences took 0.0s, 3018 sentences/s
2017-10-29 09:19:18,785 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:19:18,788 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:19:18,790 : INFO : scoring 10 sentences took 0.0s, 2894 sentences/s
2017-10-29 09:19:18,793 : INFO : scoring se

In [15]:
fraud_kw_vec = [-0.59686035,-0.6725697,-0.69134709,-0.62265558,-0.59475399,-0.69835925,
                -0.67224162,-0.67393852,-0.67712811,-0.58945206]

In [16]:
# Cosine Similarity

from scipy import spatial

ratio_fraud = 1 - spatial.distance.cosine(fraud_kw_vec[:len(letter_score[-1][0])], letter_score[-1])
print (ratio_fraud)

0.953370926155
